The purpose of this notebook is to provide a comprehensive, step-by-step tutorial for fine-tuning any LLM (Large Language Model). Unlike many tutorials available, I’ll explain each step in a detailed manner, covering all classes, functions, and parameters used.

In [1]:
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off

In [2]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

Before loading the model, we will define a function create_bnb_config to define the bitsandbytes configuration. The bitsandbytes library allows model quantization. Quantization is a technique used to compress deep learning models by reducing the number of bits used to represent their weights and activations. This compression allows for faster inference and reduced memory consumption, making it possible to deploy these models on edge devices with limited resources.

By using 4-bit transformer language models, we can achieve impressive results while significantly reducing memory and computational requirements.

Hugging Face Transformers (transformers) is closely integrated with bitsandbytes. The BitsAndBytesConfig class from the transformers library allows configuring the model quantization method.

#Parameters:

load_in_4bit: Load the model in 4-bit precision, i.e., divide memory usage by 4.

bnb_4bit_use_double_quant: Use nested quantization techniques for more memory-efficient inference at no additional cost.

bnb_4bit_quant_type: Set quantization data type. The options are either FP4 (4-bit precision), which is the default quantization data type, or NF4 (Normal Float 4), a new 4-bit data type adapted for weights that have been initialized using a normal distribution.

bnb_4bit_compute_dtype: Set the computational data type for 4-bit models. Default value: torch.float32

In [4]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [5]:
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [6]:
################################################################################
# transformers parameters
################################################################################

# The pre-trained model from the Hugging Face Hub to load and fine-tune
model_name = "meta-llama/Llama-2-7b-hf"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

Load model from Hugging Face Hub with model name and bitsandbytes configuration

In [7]:
bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(model_name, bnb_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


#Loading DataSet
Now that we have loaded the Llama-2-7B model and its tokenizer, we will move on to loading our news classification instruction dataset from the previous blog as a Hugging Face Datasets.

Firstly, we will initialize the path of the dataset. In this case, we have a CSV file that contains 99 records, or prompts. This dataset contains an instruction column containing the instruction to categorize a news article into 18 categories, an input column containing the news article, and an output column containing the actual news category for training.

In [8]:
# The instruction dataset to use
dataset_name = "/content/drive/MyDrive/news_classification.csv"
# Load dataset
dataset = load_dataset("csv", data_files = dataset_name, split = "train")
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Number of prompts: 100
Column names are: ['instruction', 'input', 'output']


In [9]:
#Converting CSV To dict of prompts
dataset[randrange(len(dataset))]

{'instruction': 'Categorize the news article into one of the 18 categories:\n\nWORLD NEWS\nCOMEDY\nPOLITICS\nTECH\nSPORTS\nBUSINESS\nOTHERS\nENTERTAINMENT\nCULTURE & ARTS\nFOOD & DRINK\nMEDIA\nRELIGION\nMONEY\nHEALTHY LIVING\nSCIENCE\nEDUCATION\nCRIME\nENVIRONMENT\n\n',
 'input': 'WASHINGTON, Sept. 21, 2015 /PRNewswire-USNewswire/ -- More than 3,700 key local government decision makers, exhibitors, and guests from cities, towns, counties, related entities, and private-sectors firms throughout the world are expected in Seattle, Washington, September 27-30, to attend the 101st Annual Conference of ICMA, the International City/County Management Association.\n\nView photo\n\n"ICMA\'s Annual Conference is the premiere educational event for local government professionals offered anywhere in the world," said 2014-15 ICMA President James Bennett, city manager, Biddeford, Maine. "We have put together an outstanding program of keynotes, featured speakers, sessions, demonstrations, workshops, tou

In [10]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes task. Write a response that appropriately completes the task."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [11]:
create_prompt_formats(dataset[randrange(len(dataset))])

{'instruction': 'Categorize the news article into one of the 18 categories:\n\nWORLD NEWS\nCOMEDY\nPOLITICS\nTECH\nSPORTS\nBUSINESS\nOTHERS\nENTERTAINMENT\nCULTURE & ARTS\nFOOD & DRINK\nMEDIA\nRELIGION\nMONEY\nHEALTHY LIVING\nSCIENCE\nEDUCATION\nCRIME\nENVIRONMENT\n\n',
 'input': 'We\'ve glimpsed the future, and it\'s all about Kanye. Kanye West gave a mega-interview to Vanity Fair that came out Thursday. They had a chance to talk to him just after his New York Fashion Week collection hit the runways. Here\'s what we learned: On stressing out about his collection West referred repeatedly to not sleeping for days on end, or crashing in his workspace. "I slept at the studio and I would have dreams or nightmares about the look board," he said. Hillary Clinton has one request for Kanye\'s 2020 presidential run On how his album is coming along He described it as "a sonic landscape, a two-year painting." During his show, they played a song called "Fade" from the upcoming album. "The song I p

In [13]:
#In the next cell, we will define the get_max_length function to find out the maximum sequence length of the Llama-2-7B model.
#This function will pull the model configuration and attempt to find the maximum sequence length from one of the several configuration keys that may contain it.
#If the maximum sequence length is not found, it will default to 1024.


def get_max_length(model):

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

#Tokenizing the Batch dataset

In [14]:
def preprocess_batch(batch, tokenizer, max_length):


    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )


In [16]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "input", "output", "text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [17]:
# Random seed
seed = 33

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
print(preprocessed_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 100
})


In [19]:
print(preprocessed_dataset[0])

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 3414, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 29907, 20440, 675, 278, 9763, 4274, 964, 697, 310, 278, 29871, 29896, 29947, 13997, 29901, 13, 13, 11686, 10249, 14693, 7811, 13, 3217, 2303, 29928, 29979, 13, 29925, 5607, 1806, 2965, 29903, 13, 4330, 3210, 13, 5550, 8476, 29903, 13, 29933, 3308, 8895, 1799, 13, 2891, 4448, 29903, 13, 3919, 1001, 6040, 1177, 13780, 13, 29907, 8647, 11499, 669, 9033, 9375, 13, 5800, 13668, 669, 26900, 1177, 29968, 13, 2303, 4571, 29909, 13, 1525, 5265, 29954, 2725, 13, 29924, 12413, 29979, 13, 9606, 1964, 4690, 29979, 365, 5667, 4214, 13, 7187, 29902, 1430, 4741, 13, 3352, 23129, 8098, 13, 29907, 3960, 2303, 13, 25838, 8193, 1164, 13780, 13, 13, 13, 13, 4290, 29901, 13, 29909, 1735, 310, 4423, 322, 19716, 22583, 13332, 362, 756, 1754, 278, 5032, 29834, 5127, 17847, 11367, 263, 2217, 3593, 631, 1135, 4226, 29889, 13, 13, 29933, 95